In [1]:
import numpy as np
import open3d as o3d
import math
import back_surface as back

In [6]:
def duplicateSurface(numny_array_points):
    temp = []
    for i in numny_array_points:
        temp.append([i[0],i[1],i[2]+0.08])
    return np.asarray(temp)

def displayPoints(points_list):
    temp = o3d.geometry.PointCloud()
    temp.points = o3d.utility.Vector3dVector(np.asarray(points_list))
    o3d.visualization.draw_geometries([temp])

def getCenterPoint(points_list):
    count = len(points_list)
    #init
    center = np.zeros(3)
    for i in range(count):
        center[0] += points_list[i][0]
        center[1] += points_list[i][1]
        center[2] += points_list[i][2]
    return center/count

In [2]:
pcd = o3d.io.read_point_cloud("./Points/clean points/dwnsample_pose_1.ply")

In [14]:
# lật ngược các điểm
points = np.asarray(pcd.points)
result = back.groupYAxis(points,0.1)
displayPoints(np.append(result[6],result[4],axis=0))

TypeError: concatenate() got multiple values for argument 'axis'

##search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
cam = np.array([0,0,0])
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5, max_nn=9))
pcd.orient_normals_towards_camera_location(camera_location=cam)
o3d.visualization.draw_geometries([pcd])

temp = np.asarray(pcd.points)
new_points = duplicateSurface(temp)
new_pcl = o3d.geometry.PointCloud()
new_pcl.points = o3d.utility.Vector3dVector(np.concatenate((temp,new_points)))
new_pcl.normals = o3d.utility.Vector3dVector(np.concatenate((np.asarray(pcd.normals),-np.asarray(pcd.normals))))
new_pcl.paint_uniform_color([0.8, 0.8, 0.8])
#o3d.visualization.draw_geometries([new_pcl])
#o3d.io.write_point_cloud("./clean points/dupback_pose_1.ply", new_pcl)

print('run Poisson surface reconstruction')
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(new_pcl, depth=8)
print(mesh)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

o3d.io.write_triangle_mesh('./Mesh/Poisson Mesh/pose_1.ply',mesh)